In [3]:
import time
import pandas as pd

In [4]:
# Selenium imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
#     chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, chrome_options=chrome_options)
    
    return browser

In [46]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time, verbose):
    
    # initialize webdriver
    browser = init_driver(path='../driver/chromedriver.exe') # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
#     accept_cookies = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))).click()

    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
#     time.sleep(sleep_time)
    time.sleep(3)
    jobs = []
    
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
            try:
                salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1

            if verbose:
                print(f"Job Title: {job_nu_naam}")
                print(f"Salary Estimate: {salart_estimate}")
                print(f"Job Description: {job_description}")
                print(f"Rating: {rating}")
                print(f"Company Name: {company_name}")
                print(f"Location: {location}")

            #Going to the Company tab...
            try:
                browser.find_element_by_xpath('.//div[@data-tab-type="overview"]//span[text()="Company"]//following-sibling::*').click()
                try:
                    headquarters = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[@text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = browser.find_element_by_xpath('.div[@class="infoEntity"]//label[@text()="Founded"]//following-siblings::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

            if verbose:
                print(f"Headquaters: {headquarters}")
                print(f"Size: {size}")
                print(f"Founded: {founded}")
                print(f"Type of Ownership: {type_of_ownership}")
                print(f"Industry: {industry}")
                print(f"Sector:{sector}")
                print(f"Revenue: {revenue}")
                print(f"Competitors: {competitors}")
                print("#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$")

            jobs.append({'Job Title':job_nu_naam,
                        "Salary Estimate" : salary_estimate,
                        "Job Description" : job_description,
                        "Rating" : rating,
                        "Company Name" : company_name,
                        "Location" : location,
                        "Headquarters" : headquarters,
                        "Size" : size,
                        "Founded" : founded,
                        "Type of ownership" : type_of_ownership,
                        "Industry" : industry,
                        "Sector" : sector,
                        "Revenue" : revenue,
                        "Competitors" : competitors})
        try:
            browser.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print(f"Scraping terminated before reaching target of jobs. Needed {num_jobs} , got {len(jobs)}")

    return pd.DataFrame(jobs)


In [47]:
df = search_jobs(job_name='data scientist/data science', job_location='London, England', num_jobs=1001, sleep_time=15, verbose=False)
df

C:\Users\Saurabh\anaconda3\envs\webscrape\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


 x out worked
Progress: 0/1001
Progress: 1/1001
Progress: 2/1001
Progress: 3/1001
Progress: 4/1001
Progress: 5/1001
Progress: 6/1001
Progress: 7/1001
Progress: 8/1001
Progress: 9/1001
Progress: 10/1001
Progress: 11/1001
Progress: 12/1001
Progress: 13/1001
Progress: 14/1001
Progress: 15/1001
Progress: 16/1001
Progress: 17/1001
Progress: 18/1001
Progress: 19/1001
Progress: 20/1001
Progress: 21/1001
Progress: 22/1001
Progress: 23/1001
Progress: 24/1001
Progress: 25/1001
Progress: 26/1001
Progress: 27/1001
Progress: 28/1001
Progress: 29/1001
 x out failed
Progress: 30/1001
Progress: 31/1001
Progress: 32/1001
Progress: 33/1001
Progress: 34/1001
Progress: 35/1001
Progress: 36/1001
Progress: 37/1001
Progress: 38/1001
Progress: 39/1001
Progress: 40/1001
Progress: 41/1001
Progress: 42/1001
Progress: 43/1001
Progress: 44/1001
Progress: 45/1001
Progress: 46/1001
Progress: 47/1001
Progress: 48/1001
Progress: 49/1001
Progress: 50/1001
Progress: 51/1001
Progress: 52/1001
Progress: 53/1001
Progress: 

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Science,£50K - £63K (Glassdoor est.),Love using technology to solve complex problem...,4.0,MThree Consulting\n4.0,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
1,Data Scientist - Applied Science,£40K - £51K (Glassdoor est.),Data Scientist - Applied Science\n\n\n\nJob de...,4.5,causaLens\n4.5,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
2,Data Engineer,£40K - £59K (Glassdoor est.),Job Description\n\nData Engineer\n\nAt Citadel...,3.9,Citadel\n3.9,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
3,Data Analyst,£32K - £69K (Glassdoor est.),The Tech team at gohenry is looking to recruit...,4.6,gohenry Ltd\n4.6,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
4,Data/Marketing Analyst,£32K - £59K (Glassdoor est.),Data / Marketing Analyst\n\nThe Marketing team...,4.6,gohenry Ltd\n4.6,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,Account Executive - Sub Saharan Africa,£70K - £91K (Glassdoor est.),"About Us\n\nAt Cloudflare, we have our eyes se...",3.8,Cloudflare\n3.8,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
997,Enterprise Account Executive - Iberia,£70K - £91K (Glassdoor est.),"About Us\n\nAt Cloudflare, we have our eyes se...",3.8,Cloudflare\n3.8,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
998,Regulatory Affairs Manager,£70K - £91K (Glassdoor est.),Gilead Sciences is continuing to hire for all ...,3.7,Gilead Sciences\n3.7,"Uxbridge, England",-1,-1,-1,-1,-1,-1,-1,-1
999,Mid-Market Account Executive - CEE + Russia,£70K - £91K (Glassdoor est.),"About Us\n\nAt Cloudflare, we have our eyes se...",3.8,Cloudflare\n3.8,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
